# AI-Training to play Mario

In [14]:
%pip install gym-super-mario-bros==7.4.0
%pip install tensordict==0.2.0
%pip install torchrl==0.2.0


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os

# Gym is an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

# Super Mario environment for OpenAI Gym
import gym_super_mario_bros

from tensordict import TensorDict
from torchrl.data import TensorDictReplayBuffer, LazyMemmapStorage


ImportError: DLL load failed while importing _torchrl: Die angegebene Prozedur wurde nicht gefunden.

## Definitions of RL

**Environment**: The world that an agent interacts and learns from.

**Action** $a$: How our agent responds to the environment. A set of possible actions can be called action-space.

**State** $s$: The current characteristic of the environment. This is a set of all possible states the environment can be in is called state-space.

**Reward** $r$: Reward is the key feedback from Environment to agent. It is what drives our agent to learn and change its future action. An aggreation of rewards over multiple time steps is called **Return**.

**Action value function** $Q^*(s,a)$: This models the expected return starting from a given state and the agent making a particular action. Given a starting state $s$ takes an arbitrary function $a$, and then for each future time step take the action that maximises returns. The $Q$ stands for the quality of an action in a state.


## Environment

### Initialisation of the environment

In Mario, the environment consists of tubes, mushrooms, enemies and other components.

When Mario makes an action, the environment responds with the changed (next) state, reward and other info.

Here is the Super Mario environment initialised (in v0.26 we change the render_mode to 'rgb' to see the results on the screen):

In [17]:
if gym.__version__ < '0.26':
    env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0', new_step_api=True)
else:
    env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0', render_mode='rgb', apply_api_compatibility=True)

Limits the action space to:

0. walk right
1. jump right


In [18]:
env = JoypadSpace(env, [['right'], ['right', 'A']])

env.reset()
next_state, reward, done, trunc, info = env.step(action=0)
print(f"{next_state.shape}, {reward}, {done}, {trunc}, {info}")

(240, 256, 3), 0.0, False, False, {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 79}
